In [41]:
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
seq2seq_model1 = load_model('nmt.h5')

In [42]:
import pandas as pd
lines= pd.read_table('fra.txt', names=['eng', 'fra'],index_col=False,nrows = 200000)

C:\Users\Asus\AppData\Local\Temp\ipykernel_21156\3335428549.py:2: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  lines= pd.read_table('fra.txt', names=['eng', 'fra'],index_col=False,nrows = 200000)


In [43]:
import string
lines.eng = lines.eng.apply(lambda x:x.lower())
lines.fra = lines.fra.apply(lambda x:x.lower())
punctuation = set(string.punctuation)
lines.eng = lines.eng.apply(lambda x:''.join(char for char in x if char not in punctuation))
lines.fra = lines.fra.apply(lambda x:''.join(char for char in x if char not in punctuation))

In [44]:
digits = set(string.digits)
lines.eng = lines.eng.apply(lambda x:''.join(char for char in x if char not in digits))
lines.fra = lines.fra.apply(lambda x:''.join(char for char in x if char not in digits))

In [45]:
import re
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.fra=lines.fra.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.fra=lines.fra.apply(lambda x: re.sub(" +", " ", x))

In [46]:
lines.fra = lines.fra.apply(lambda x : '<SOS> '+ x + ' <EOS>')

In [47]:
english_words = set()
for line in lines.eng:
    for word in line.split():
        if word not in english_words:
            english_words.add(word)
french_words = set()
for line in lines.fra:
    for word in line.split():
        if word not in french_words:
            french_words.add(word)            

In [48]:
total_english_words = len(english_words)
total_french_words = len(french_words)
total_english_words,total_french_words

(13720, 28302)

In [49]:
import numpy as np
eng_len = []
for line in lines.eng:
    eng_len.append(len(line.split()))
max_english_sen_len = np.max(eng_len)
max_english_sen_len

12

In [50]:
fra_len = []
for line in lines.fra:
    fra_len.append(len(line.split()))
max_french_sen_len = np.max(fra_len)
max_french_sen_len

22

In [51]:
input_words = sorted(list(english_words))
target_words = sorted(list(french_words))
num_encoder_tokens = len(english_words)+1
num_decoder_tokens = len(french_words)+1
num_decoder_tokens+=1 #zero padding
num_encoder_tokens, num_decoder_tokens

(13721, 28304)

In [52]:
from sklearn.model_selection import train_test_split
x, y = lines.eng, lines.fra
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
x_train.shape, x_test.shape

((160000,), (40000,))

In [53]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [54]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [73]:
import pickle
with open("input_token_index.pkl", "wb") as f:
    pickle.dump(input_token_index, f)
with open("target_token_index.pkl", "wb") as f:
    pickle.dump(target_token_index, f)    
with open("reverse_target_char_index.pkl", "wb") as f:
    pickle.dump(reverse_target_char_index, f)      

In [70]:
def translate(input_sentence):
    # Preprocess input sentence
    input_sentence = input_sentence.lower()
    input_sentence = ''.join(char for char in input_sentence if char not in punctuation)
    input_sentence = ''.join(char for char in input_sentence if char not in digits)
    input_sentence = input_sentence.strip()
    input_sentence = re.sub(" +", " ", input_sentence)
    
    # Tokenize input sentence
    input_words = input_sentence.split()
    input_indices = [input_token_index.get(word, 0) for word in input_words]
    print(input_indices)
    # Create encoder input
    encoder_input_data = np.zeros((1, max_english_sen_len), dtype='float32')
    for t, index in enumerate(input_indices):
        encoder_input_data[0, t] = index
    
    # Initial decoder input
    decoder_input_data = np.zeros((1, 1), dtype='float32')
    decoder_input_data[0, 0] = target_token_index['<SOS>']
    
    # Translate using the model
    translation = ''
    for _ in range(max_french_sen_len):
        decoder_output = seq2seq_model1.predict([encoder_input_data, decoder_input_data])
        predicted_token_index = np.argmax(decoder_output[0, -1, :])
        print(predicted_token_index)
        predicted_word = reverse_target_char_index.get(predicted_token_index, '<UNK>')
        
        if predicted_word == '<EOS>':
            break
        
        translation += predicted_word + ' '
        
        # Update decoder input for next iteration
        decoder_input_data = np.zeros((1, 1), dtype='float32')
        decoder_input_data[0, 0] = predicted_token_index
    
    return translation.strip()

input_sentence = "i am a boy"
predicted_translation = translate(input_sentence)
word_count = len(input_sentence.split())
words = len(predicted_translation.split())
prediction = ""
i = 0
for word in predicted_translation.split():
    prediction+=word
    prediction+=" "
    i+=1
    if i==word_count:
        break

print("Input:", input_sentence)
print("Predicted Translation:", prediction)

[5981, 375, 1, 1434]
1/1 [==============================] - 0s 21ms/step
12662
1/1 [==============================] - 0s 22ms/step
24578
1/1 [==============================] - 0s 22ms/step
26640
1/1 [==============================] - 0s 23ms/step
10695
1/1 [==============================] - 0s 23ms/step
20474
1/1 [==============================] - 0s 22ms/step
12662
1/1 [==============================] - 0s 22ms/step
24578
1/1 [==============================] - 0s 23ms/step
26640
1/1 [==============================] - 0s 22ms/step
10695
1/1 [==============================] - 0s 21ms/step
20474
1/1 [==============================] - 0s 22ms/step
12662
1/1 [==============================] - 0s 22ms/step
24578
1/1 [==============================] - 0s 22ms/step
26640
1/1 [==============================] - 0s 25ms/step
10695
1/1 [==============================] - 0s 23ms/step
20474
1/1 [==============================] - 0s 22ms/step
12662
1/1 [==============================] - 0s 22ms/step